<a href="https://colab.research.google.com/github/Mansari/pairings-of-beautiful-names-of-allah-in-quran/blob/main/pairings_of_beautiful_names_of_allah_in_quran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Pairings of Allah's 99 Names in the Quranic Corpus

> NOTE: This project is a preliminary exploration driven by my personal interest in gaining a deeper understanding of the Quran. As I am not a scholar, please review the contents critically and apply your own judgment in its interpretation and use.

## Background

My fascination with the pairings of Allah SWT's beautiful names in the Quran has driven me to delve deeper into their significance. Consider, for instance, the pairing of السميع العليم (As-Sami' Al-Alim) in Surat An-Nisa, Aya 220 ([26:220](https://quran.com/26/220)). What profound insights might we glean from the deliberate coupling of these specific names by Allah SWT? This notebook documents my journey to programmatically identify and analyze these pairings.

## Usage

Designed as both a tutorial and a tool, this notebook leverages open-source resources to systematically analyze the Quranic corpus using Python. While executing the entire notebook produces a file with the final results, I encourage a step-by-step walkthrough to fully grasp the underlying process.

## License

This code is open for use in personal projects, adhering to the stipulations of the [Quran CSV](https://github.com/azvox/quran-csv/blob/master/resources/README.md) license. Your adherence to these guidelines is appreciated.

## Contact

For any queries or discussions, feel free to reach out to me via [email](mailto:mohammad.ansari.ca@gmail.com) or connect on [LinkedIn](https://www.linkedin.com)


## The Journey Begins

Our first step in this exploratory journey involves acquiring the foundational data. We will start by downloading the Quran corpus in CSV format. This critical resource is made available through the generosity of contributors on the [Quran CSV Github](https://github.com/azvox/quran-csv/)


### Downloading the Quran Corpus
To download the Quran corpus in CSV format, execute the following command:


In [ ]:
!wget "https://raw.githubusercontent.com/azvox/quran-csv/master/resources/arabic-default.csv"
print("Quran text (CSV) successfully downloaded.")

### Acquiring the 99 Attributes of Allah SWT

Next, we enrich our dataset with another crucial element: the 99 attributes of Allah SWT. These are sourced from a dedicated repository, which has compiled them with care and respect. You can access this resource at the [99 Names Of Allah Github repository](https://github.com/KabDeveloper/99-Names-Of-Allah).

To download this data, use the following command:

In [ ]:
!wget "https://raw.githubusercontent.com/KabDeveloper/99-Names-Of-Allah/master/99_Names_Of_Allah.json"
print("99 Names of Allah (CSV) successfully downloaded.")

### Installing PyArabic for Text Processing

To enhance our text analysis capabilities, we'll install the [PyArabic](https://pypi.org/project/PyArabic/) library. PyArabic is instrumental in processing Arabic text, particularly in removing diacritics (tashkeel), which simplifies searching and analysis. This step is crucial for accurately parsing and understanding the Quranic text in our study.

In [ ]:
!pip install PyArabic
print("Libraries installed.")

### Loading the Names of Allah SWT

Having downloaded the data, we now turn our attention to loading the beautiful names of Allah SWT from the JSON file we acquired earlier. This step is crucial for integrating these names into our analysis.

The following code snippet will load the names and display a preview of the data by showing a few rows:

In [ ]:
import pandas as pd

# Read the JSON file into a df
attributes = pd.read_json('99_Names_Of_Allah.json', orient='records')

# Normalize the data object into a DF
attributes = pd.json_normalize(attributes["data"])

# Print the first few rows of the DataFrame
print(attributes.head())

### Preprocessing the Attributes List

Before diving into the analysis, it's essential to preprocess the list of Allah SWT's attributes. This preprocessing step involves creating variations of each name that might appear in the Quranic text. Such variations account for different possible forms, ensuring our search and analysis are comprehensive and accurate.


In [ ]:
import pyarabic.araby as araby

# الرَّحِيمُ --> الرحيم
attributes["name_no_diacritics"] = attributes["name"].apply(
    lambda x: araby.strip_diacritics(x)
)

# الرحيم --> رحيم
attributes["name_no_leading_al"] = attributes["name_no_diacritics"].apply(
    lambda x: x[2:] if x.startswith('ال') else x
)

# رحيم --> رحيما
attributes["name_added_alef"] = attributes["name_no_leading_al"].apply(
    lambda x: x + "ا"
)

# Print the first few rows of the DataFrame
print(attributes.head())

### Generating Pairings from the Preprocessed Names

After preparing the variations of Allah SWT's attributes, we move to generate all possible pairings from these variations. This crucial step involves creating combinations of the names to reflect their potential pairings as found in the Quran. It's a comprehensive approach to understand how the attributes of Allah are interconnected and presented in various contexts within the Quranic scripture.

By generating these pairings, we lay the groundwork for a detailed exploration and analysis of their occurrences and significance throughout the Quran.

In [ ]:
# define a function to generate combinations from a list with added space in between
def generate_pairings(list):
  combinations = []
  for i in range(len(list)):
    for j in range(len(list)):
      # avoid generation combinations of the same name
      if (i != j):
        combinations.append(list[i] + ' ' + list[j])
  return combinations

# execute on the 3 variation of attributes/names
beautiful_names_pairs = []
beautiful_names_pairs.extend(generate_pairings(attributes["name_no_diacritics"]))
beautiful_names_pairs.extend(generate_pairings(attributes["name_no_leading_al"]))
beautiful_names_pairs.extend(generate_pairings(attributes["name_added_alef"]))

print("Generated " + str(len(beautiful_names_pairs)) + " pairings.")
print(beautiful_names_pairs[:5])

### Loading and Processing the Quran Corpus

The next phase in our analysis involves loading the content of the Quran corpus. Once loaded, we will create an additional field in our dataset specifically for the text of each verse, but with a significant modification: the removal of diacritics. This step is crucial as it simplifies the text for easier matching with our preprocessed list of Allah's attributes. By doing so, we enhance the accuracy and efficiency of our search for attribute pairings within the Quranic text.

In [ ]:
# Load the CSV file into a pandas dataframe
quran_df = pd.read_csv('arabic-default.csv')

# Remove Tashkeel (Diactrictics) from all verses for easier search
quran_df["content_ar"] = quran_df["content_ar"].apply(
    lambda x: araby.strip_diacritics(x)
)

# This Quran text contains leading basmallah before the first verse in each surah
# To avoid matchint the attributes Rahman and Raheem in every surah.
# i.e.: "بسم الله الرحمن الرحيم الم"
# We will then remove the leading " بسم الله الرحمن الرحيم" (with space) from any verses
quran_df["content_ar"] = quran_df["content_ar"].str.replace("بسم الله الرحمن الرحيم ","")

print("Quran arabic content loaded and pre-processed. Sample below:")

# Print the first 5 lines as an example
print(quran_df.head(10))

### Filtering Verses Using Pandas Preprocessing

Our analysis now takes an intriguing turn with the application of a preprocessing technique using pandas. This method will enable us to filter the verses of the Quran, focusing specifically on those that contain any combination of the beautiful names pairings. By applying this technique, we effectively narrow down our corpus to the most relevant verses, allowing for a more targeted and meaningful analysis of how these pairings are used throughout the Quranic text.

In [ ]:
# Use the str.contains() method to search for the words in the Name and
# City columns of the dataframe. The \b forces the exact match of the pairing
# (issus with سميع علي which is a valid combo matching سميع عليم)
# BUG ALERT - if you use \b in the beginning and end, it will not capture
# words where the beginning is ل for example لغفور رحيم
# so we only include \b for an exact match towards the end of the pair
matching_pattern = '(' + '|'.join(beautiful_names_pairs) + r')\b'
#matching_pattern = ('|'.join(beautiful_names_pairs))
mask = quran_df['content_ar'].str.contains(matching_pattern, case=False)

# Filter the dataframe based on the mask
filtered_df = quran_df[mask]

print("Processed " + str(len(quran_df)) + " verses and matched " + str(len(filtered_df)) + " verses.")

### Conducting the Search for Pairings

We now embark on the most intensive phase of our project: the search through the Quran corpus for the pairings of Allah's names. Given the vast number of combinations – over 29,000 possible pairings – this process can be quite time-consuming. However, it's a crucial step in our endeavor, as it involves meticulously scouring the text to uncover how these pairings are manifested across different verses and contexts within the Quran.

In [ ]:
# Create an empty list to store the matching values
matching_pairs = []

# Loop through each row in the dataframe
for index, row in filtered_df.iterrows():
    # Check if any of the pairings are a partial match for the 'content_ar_no_diacritics' column of the current row
    for pair in beautiful_names_pairs:
        if pair in row['content_ar']:
            # If a match is found, append the value of the 'other_column' to the matching_values list
            matching_pairs.append({'pairing': pair, 'verse': row['content_ar'], 'chapter': row['chapter_number'], 'ayah': row['Ayah_number']})
            break

print("Processing completed for " + str(len(matching_pairs)) + " pairings.")

### Inspecting the Results

After completing the extensive search, we focus on examining the findings. To provide a glimpse into our results, we showcase a brief preview of the top 5 pairings.

In [ ]:
for i in range(5):
    item = matching_pairs[i]
    print("Item ", i+1, ":")
    for key, value in item.items():
        print(key, ":", value)
    print()

### Exporting Results to CSV

With our findings compiled, the next step is to export these results to a CSV file. This format will facilitate further analysis and sharing of our insights.

`TODO` Consider enhancing the analysis process by using pandas DataFrame for statistical analysis directly within this notebook. This approach could streamline the process, eliminating the need to export raw data to a CSV and then analyze it externally in Sheets.

In [ ]:
from google.colab import files
import csv

with open('99_names_of_Allah_swt_pairing_analysis_results.csv', 'w', newline='') as csvfile:
    fieldnames = ['pairing', 'verse', 'chapter', 'ayah']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for obj in matching_pairs:
        writer.writerow(obj)

print("Resuls exported!")

### Downloading for External Analysis

Once the results are exported to a CSV file, you have the option to download this file for additional analysis using tools like Google Sheets. This step allows for a more flexible and detailed examination of the data, leveraging the advanced features of spreadsheet software to draw deeper insights and conclusions from our findings.

In [ ]:
# Downoad the file locally
files.download('99_names_of_Allah_swt_pairing_analysis_results.csv')

### Conclusion and Next Steps

That's it! Jazak Allahu Khairan (JAK) for reading and engaging with this project. Alhamdulillah, we have reached a significant milestone in our exploration of the Quran.

The journey doesn't end here, though. The next step involves delving into the books of tafsir (interpretations of the Quran) and analyzing the pairings in the context of the verses that include them. This deeper exploration will help us understand the significance of these pairings and the profound messages they convey within the Quran.

Thank you for joining me on this enlightening journey. May our continued efforts bring us closer to comprehending the wisdom of the Quran.